<a href="https://colab.research.google.com/github/noahdanieldsouza/PAM-classification/blob/main/HDBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchaudio numpy
!pip install git+https://github.com/google-research/perch-hoplite.git@782acd0e409eb27df51a695de4cb6608dae0db25

In [ ]:
import os
import torch
import torchaudio
import numpy as np
import csv
import gc
import tempfile

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from perch_hoplite.agile import colab_utils, embed, source_info
from perch_hoplite.db import sqlite_usearch_impl
from perch_hoplite.zoo import model_configs
from perch_hoplite.agile.classifier import LinearClassifier

# --- Paths ---
db_path = '/content/drive/My Drive/classifier_training/DB'
classifier_path = '/content/drive/MyDrive/full_labeled_fish/DB/agile_classifier_v2.pt'


# --- Load DB + model ---
db = sqlite_usearch_impl.SQLiteUsearchDB.create(db_path)
db_model_config = db.get_metadata('model_config')
model_class = model_configs.get_model_class(db_model_config['model_key'])
embedding_model = model_class.from_config(db_model_config['model_config'])
embedding_ids = db.get_embedding_ids()
print(f"✅ loaded {len(embedding_ids)} embeddings")
id = embedding_ids[0]
vector = db.get_embedding(id)
print(f"✅ vector {vector}")

# --- Load classifier ---
classifier = LinearClassifier.load(classifier_path)
class_names = classifier.classes
print("✅ Loaded classifier with classes:", class_names)

train_set = []

limit = 0;
for emb_id in embedding_ids:
    limit += 1
    if limit > 30000:
        break
    print (f"processing embedding: {emb_id}")
    vector = db.get_embedding(emb_id)
    print(vector.shape)

    train_set.append(vector)




print(np.shape(train_set))








In [ ]:
import numpy as np

# Suppose this is your list of vectors
# train_set = [array([..1280..]), array([..1280..]), ...]

X = np.array(train_set)
print(X.shape)   # (21600, 1280)

In [2]:
from sklearn.decomposition import PCA

#pca = PCA(n_components=90)  # or '0.95' for 95% explained variance
#X_pca = pca.fit_transform(X_scaled)


In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=20,       # avoid fragmenting noise
    min_samples=10,            # enforces strong density
    cluster_selection_method='eom',  # better for noisy data
    prediction_data=True)
labels = clusterer.fit_predict(X_umap)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(8,6))
scatter = plt.scatter(X_umap[:,0], X_umap[:,1], c=labels, cmap='Spectral', s=5)

# Create legend manually
unique_labels = np.unique(labels)
print(len(unique_labels) )
for cluster_id in unique_labels:
    if cluster_id == -1:
        label_name = "Noise"
    else:
        label_name = f"Cluster {cluster_id}"
plt.legend(handles=scatter.legend_elements()[0], labels=[f"Cluster {l}" for l in unique_labels], bbox_to_anchor=(1.05, 1))
plt.show()



In [8]:
cursor = db._get_cursor()

In [ ]:
import os
import random
import numpy as np
import librosa
import IPython.display as ipd

# Define your base path for audio files
base_path = "/content/drive/MyDrive/mass_data/cut2"
base_path2 = "/content/drive/MyDrive/mass_data/cut1"
base_path3 = "/content/drive/MyDrive/mass_data/cut"

# Get all unique clusters (excluding noise if present)
unique_labels = np.unique(labels)


# Loop through each cluster
for cluster_id in unique_labels:
    cluster_indices = np.where(labels == cluster_id)[0]
    print(f"\n🎯 Cluster {cluster_id} — {len(cluster_indices)} embeddings")

    # Skip clusters with fewer than 2 samples
    if len(cluster_indices) < 2:
        print("Not enough samples to display (skipping)")
        continue

    # Randomly choose 2 embeddings from this cluster
    sample_indices = random.sample(list(cluster_indices), 2)

    for idx in sample_indices:
        embedding_id = int(embedding_ids[idx])

        # Query database for metadata
        cursor.execute(
            """
            SELECT
              he.id,
              hs.source,
              hs.dataset,
              he.offsets
            FROM hoplite_embeddings he
            JOIN hoplite_sources hs ON he.source_idx = hs.id
            WHERE he.id = ?;
            """,
            (embedding_id,),
        )
        row = cursor.fetchone()

        if not row:
            print(f"⚠️ No entry found for embedding_id {embedding_id}")
            continue

        _, source, dataset, _ = row
        file_path = os.path.join(base_path, source)

        if not os.path.exists(file_path):
            file_path = os.path.join(base_path2, source)
            if not os.path.exists(file_path):
              file_path = os.path.join(base_path3, source)
              if not os.path.exists(file_path):
                print(f"❌ File not found: {file_path}")
                continue

        print(f"\n▶️ Cluster {cluster_id}: {source}")
        y, sr = librosa.load(file_path, sr=None)
        display(ipd.Audio(y, rate=sr))


In [ ]:
import joblib

joblib.dump(
    {
        "umap": reducer,       # your trained UMAP reducer
        "clusterer": clusterer # your trained HDBSCAN clusterer
    },
    "/content/drive/MyDrive/umap_clusterer_model.joblib"
)



In [ ]:
model = joblib.load("/content/drive/MyDrive/umap_clusterer_model.joblib")
print(model.keys())


In [ ]:
import os
import random
import numpy as np
import librosa
import IPython.display as ipd

# Define your base path for audio files
base_path = "/content/drive/MyDrive/mass_data/cut2"
base_path2 = "/content/drive/MyDrive/mass_data/cut1"
base_path3 = "/content/drive/MyDrive/mass_data/cut"

# Get all unique clusters (excluding noise if present)
unique_labels = np.unique(labels)


# Loop through each cluster
for cluster_id in [lbl for lbl in unique_labels if lbl == -1]:
    cluster_indices = np.where(labels == cluster_id)[0]
    print(f"\n🎯 Cluster {cluster_id} — {len(cluster_indices)} embeddings")

    # Skip clusters with fewer than 2 samples
    if len(cluster_indices) < 2:
        print("Not enough samples to display (skipping)")
        continue

    # Randomly choose 2 embeddings from this cluster
    sample_indices = cluster_indices

    for idx in sample_indices:
        embedding_id = int(embedding_ids[idx])

        # Query database for metadata
        cursor.execute(
            """
            SELECT
              he.id,
              hs.source,
              hs.dataset,
              he.offsets
            FROM hoplite_embeddings he
            JOIN hoplite_sources hs ON he.source_idx = hs.id
            WHERE he.id = ?;
            """,
            (embedding_id,),
        )
        row = cursor.fetchone()

        if not row:
            print(f"⚠️ No entry found for embedding_id {embedding_id}")
            continue

        _, source, dataset, _ = row
        file_path = os.path.join(base_path, source)

        if not os.path.exists(file_path):
            file_path = os.path.join(base_path2, source)
            if not os.path.exists(file_path):
              file_path = os.path.join(base_path3, source)
              if not os.path.exists(file_path):
                print(f"❌ File not found: {file_path}")
                continue

        print(f"\n▶️ Cluster {cluster_id}: {source}")
        y, sr = librosa.load(file_path, sr=None)
        display(ipd.Audio(y, rate=sr))

In [ ]:
import random

for i in unique_labels:
    if i == 1:  # skip cluster ID 1
        continue

    cluster_indices = np.where(labels == i)[0]
    print(f"Cluster {i} has {len(cluster_indices)} embeddings")

    # Take up to 5 random samples (handles small clusters safely)
    n_samples = (len(cluster_indices))
    sample_indices = random.sample(list(cluster_indices), n_samples)

    for idx in sample_indices:
        embedding_id = int(embedding_ids[idx])

        cursor.execute(
            """
            SELECT
              he.id,
              hs.source,
              hs.dataset,
              he.offsets
            FROM hoplite_embeddings he
            JOIN hoplite_sources hs ON he.source_idx = hs.id
            WHERE he.id = ?;
            """,
            (embedding_id,),
        )

        row = cursor.fetchone()
        if row:
            _, source, dataset, offsets = row
            # Only print rows where the filename contains "chunk"
            if "chunk" in source.lower():
                print(row)
        else:
            print(f"No entry found for embedding_id {embedding_id}")


In [10]:
cluster_id = 3
mask = labels == cluster_id
X_sub = X[mask]


In [ ]:
import umap

reducer_sub = umap.UMAP(
    n_neighbors=5,
    min_dist=0.0,
    n_components=10
)
X_sub_umap = reducer_sub.fit_transform(X_sub)
print (np.shape(X_sub_umap))


In [ ]:
import hdbscan

clusterer_sub = hdbscan.HDBSCAN(
    min_cluster_size=10,
    min_samples=3,
    cluster_selection_method='eom'
)
labels_sub = clusterer_sub.fit_predict(X_sub_umap)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(8,6))
scatter = plt.scatter(X_sub_umap[:,0], X_sub_umap[:,1], c=labels_sub, cmap='Spectral', s=5)

# Create legend manually
unique_labels = np.unique(labels_sub)
print(len(unique_labels) )
for cluster_id in unique_labels:
    if cluster_id == -1:
        label_name = "Noise"
    else:
        label_name = f"Cluster {cluster_id}"
plt.legend(handles=scatter.legend_elements()[0], labels=[f"Cluster {l}" for l in unique_labels], bbox_to_anchor=(1.05, 1))
plt.show()

In [ ]:
import os
import random
import numpy as np
import librosa
import IPython.display as ipd

# Define your base path for audio files
base_path = "/content/drive/MyDrive/mass_data/cut2"
base_path2 = "/content/drive/MyDrive/mass_data/cut1"
base_path3 = "/content/drive/MyDrive/mass_data/cut"

# Get all unique clusters (excluding noise if present)
unique_labels = np.unique(labels_sub)


# Loop through each cluster
for cluster_id in unique_labels:
    cluster_indices = np.where(labels_sub == cluster_id)[0]
    print(f"\n🎯 Cluster {cluster_id} — {len(cluster_indices)} embeddings")

    # Skip clusters with fewer than 2 samples
    if len(cluster_indices) < 2:
        print("Not enough samples to display (skipping)")
        continue

    # Randomly choose 2 embeddings from this cluster
    sample_indices = cluster_indices

    for idx in sample_indices:
        embedding_id = int(embedding_ids[idx])

        # Query database for metadata
        cursor.execute(
            """
            SELECT
              he.id,
              hs.source,
              hs.dataset,
              he.offsets
            FROM hoplite_embeddings he
            JOIN hoplite_sources hs ON he.source_idx = hs.id
            WHERE he.id = ?;
            """,
            (embedding_id,),
        )
        row = cursor.fetchone()

        if not row:
            print(f"⚠️ No entry found for embedding_id {embedding_id}")
            continue

        _, source, dataset, _ = row
        file_path = os.path.join(base_path, source)

        if not os.path.exists(file_path):
            file_path = os.path.join(base_path2, source)
            if not os.path.exists(file_path):
              file_path = os.path.join(base_path3, source)
              if not os.path.exists(file_path):
                print(f"❌ File not found: {file_path}")
                continue

        print(f"\n▶️ Cluster {cluster_id}: {source}")
        y, sr = librosa.load(file_path, sr=None)
        display(ipd.Audio(y, rate=sr))